In [198]:
#this is an automatic pairing attempt with geocode, using the company and the poskod work
import pandas as pd

# Read the FOM list.xlsx file and assign it to a dataframe
fom_df = pd.read_excel('new FOM list.xlsx')

# Read the Poskod_geocoded_Kulai_Kulim.xlsx file and assign it to another dataframe
poskod_df = pd.read_excel('Poskod_geocoded_Kulai_Kulim.xlsx')

In [199]:
# fom_df.head()
selected_columns = ["address", "Latitude", "Longitude"]
subset_fom_df = fom_df[selected_columns]
print(subset_fom_df.head())

                                             address  Latitude   Longitude
0  Stellar Suites, Jln Puteri 4/7, Bandar Puteri ...  3.017443  101.614070
1  Lot 19-ACD, Level 5, Suites A, Menara Pandan B...  3.128181  101.754464
2  Cheras Business Centre Cheras Kuala Lumpur Fed...  3.100545  101.739516
3  No. 5-1, Level 5, PV128, No. 128, Jalan Gentin...  3.201094  101.717750
4  Unit 3.3A, 3rd Floor, Surian Tower, No. 1 Jala...  3.155682  101.609527


In [200]:
# poskod_df.head()
selected_columns = ["POSKOD", "ADDRESS", "CITY", "STATE","Latitude","Longitude"]
subset_fom_df = poskod_df[selected_columns]
print(subset_fom_df.head())

   POSKOD               ADDRESS    CITY   STATE  Latitude   Longitude
0    1000            Alor Redeh  Kangar  Perlis  6.440633  100.198371
1    1000       Alor Tok Terang  Kangar  Perlis  6.435442  100.192123
2    1000           Bohor Gelam  Kangar  Perlis  6.415518  100.241577
3    1000  Jalan Abi Tok Hashim  Kangar  Perlis  6.451320  100.192883
4    1000       Jalan Alor Sena  Kangar  Perlis  6.494892  100.229450


In [201]:
unique_states = poskod_df['STATE'].unique()
# print(unique_states)
# unique_states = [state if state not in {"PJY", "LBN"} else "Putrajaya" if state == "PJY" else "Labuan" for state in unique_states]
print(unique_states)

['Perlis' 'Kedah' 'Penang' 'Kelantan' 'Terengganu' 'Pahang' 'Selangor'
 'Perak' 'Kuala Lumpur' 'PJY' 'Negeri Sembilan' 'Melaka' 'Johor' 'LBN'
 'Sabah' 'Sarawak']


In [202]:
fom_df['address'] = fom_df['address'].astype(str)


# Function to find a matching state
def find_matching_state(address):
    # Split the address into parts by comma and reverse the order
    address_parts = [part.strip() for part in reversed(address.split(','))]
    for part in address_parts:
        for state in unique_states:
            if state.lower() in part.lower():
                return state
    # Handle special case for "Pulau Pinang"
    if "pulau pinang" in address.lower():
        return "Penang"
    # Handle special case for "Melaka"
    if "malacca" in address.lower():
        return "Melaka"
    # Handle special case for "Putrajaya"
    if "putrajaya" in address.lower():
        return "PJY"
    return None

# Apply the function to the 'address' column to find matching states
fom_df['STATE'] = fom_df['address'].apply(find_matching_state)

# Display the first few rows of the updated fom_df dataframe
# print(fom_df.head())


In [203]:
# # Save the DataFrame to an Excel file
# fom_df.to_excel('fom_df_output.xlsx', index=False)

In [204]:
unique_cities = poskod_df['CITY'].unique()
# unique_states = [state if state not in {"PJY", "LBN"} else "Putrajaya" if state == "PJY" else "Labuan" for state in unique_states]
# print(unique_cities)

In [205]:
# Function to find a matching state
def find_matching_cities(address, state_to_match):
    if pd.notna(state_to_match):  # Check if the 'State' column has a value
        # Split the address into parts by comma and reverse the order
        address_parts = [part.strip() for part in reversed(address.split(','))]
        for part in address_parts:
            for state in unique_cities:
                if state.lower() in part.lower():
                    return state
    #     # Handle special case for "Pulau Pinang"
    #     if "pulau pinang" in address.lower():
    #         return "Penang"
    #     # Handle special case for "Melaka"
    #     if "malacca" in address.lower():
    #         return "Melaka"
        return None

# Apply the function to the 'address' column to find matching states
fom_df['CITY'] = fom_df.apply(lambda row: find_matching_cities(row['address'], row['STATE']), axis=1)

# # Reorder the columns to place "City" behind "State"
# fom_df2 = fom_df[['address','CITY','STATE']]

# # Display the first few rows of the updated fom_df dataframe
# print(fom_df2.head())


In [206]:
# # Save the DataFrame to an Excel file
# fom_df.to_excel('fom_df_output.xlsx', index=False)

## this reshuffles the columns

In [207]:
fom_df3 = fom_df

# Define the column names in the desired order
desired_order = ['address', 'CITY', 'STATE']

# Create a list of all columns in the original order
all_columns = list(fom_df3.columns)

# Reorder the columns while keeping the original order for other columns
new_column_order = desired_order + [col for col in all_columns if col not in desired_order]
fom_df3 = fom_df3[new_column_order]

# Display the updated fom_df dataframe
# print(fom_df3.head())

# Save the DataFrame to an Excel file
fom_df3.to_excel('fom_df3.xlsx', index=False)

In [208]:
## for those with STATE and CITY try comparing the geocode
# print(fom_df3.head())
#poskod_df

## This is where we manually edit and fix the problematic addresses. The ones with wrong states, the missing ones, too short ones, etc

In [220]:
# Read the FOM list.xlsx file and assign it to a dataframe
fom_df3 = pd.read_excel('fom_df3.xlsx')

In [221]:
from geopy.distance import geodesic

# Assuming you have two dataframes: fom_df3 and poskod_df

# Define a function to find the closest row in poskod_df for a given row in fom_df3
def find_closest(row):
    fom_location = (row['Latitude'], row['Longitude'])
    poskod_subset = poskod_df[(poskod_df['CITY'] == row['CITY']) & (poskod_df['STATE'] == row['STATE'])
                            & (poskod_df['Latitude'].notnull()) & (poskod_df['Longitude'].notnull())]
    
    if poskod_subset.empty:
        return pd.Series([None, None, None, None], index=['pair_ADDRESS', 'pair_POSKOD', 'pair_CITY', 'pair_STATE'])
    
    poskod_subset['distance'] = poskod_subset.apply(
        lambda poskod_row: geodesic(fom_location, (poskod_row['Latitude'], poskod_row['Longitude'])).miles,
        axis=1
    )
    
    closest_row = poskod_subset.loc[poskod_subset['distance'].idxmin()]
    
    return pd.Series([closest_row['ADDRESS'], closest_row['POSKOD'], closest_row['CITY'], closest_row['STATE']],
                     index=['pair_ADDRESS', 'pair_POSKOD', 'pair_CITY', 'pair_STATE'])

# Apply the function to each row in fom_df3 and concatenate the results
fom_df3[['pair_ADDRESS', 'pair_POSKOD', 'pair_CITY', 'pair_STATE']] = fom_df3.apply(find_closest, axis=1)

# Remove the 'distance' column if you added it to the poskod_df
if 'distance' in poskod_df.columns:
    poskod_df.drop('distance', axis=1, inplace=True)

In [222]:
print(fom_df3.head())

                                             address           CITY  \
0  Stellar Suites, Jln Puteri 4/7, Bandar Puteri ...        Puchong   
1  Lot 19-ACD, Level 5, Suites A, Menara Pandan B...   Kuala Lumpur   
2  Cheras Business Centre Cheras Kuala Lumpur Fed...         Cheras   
3  No. 5-1, Level 5, PV128, No. 128, Jalan Gentin...   Kuala Lumpur   
4  Unit 3.3A, 3rd Floor, Surian Tower, No. 1 Jala...  Petaling Jaya   

          STATE                          Unique_ORG_NAME    company_id  \
0      Selangor  1Training Worklife Management Sdn. Bhd.  1.650000e+14   
1  Kuala Lumpur                 Abbaz Advisory Sdn. Bhd.  8.625140e+05   
2  Kuala Lumpur           AC SUCCESS CONSULTANTS SDN BHD  9.860750e+05   
3  Kuala Lumpur                               Adam & Co.  6.149150e+05   
4      Selangor        Advent MS Tax Consultants Sdn Bhd           NaN   

                              company_name  Latitude   Longitude  \
0  1TRAINING WORKLIFE MANAGEMENT SDN. BHD.  3.017443  101.61

In [223]:
# Save the DataFrame to an Excel file
fom_df3.to_excel('fom_df4.xlsx', index=False)